In [64]:
import pandas as pd
import time
from datetime import timedelta
import math
import numpy as np    

# Cleaning the dataset

In [2]:
df = pd.read_csv('delivery_orders_march.csv')

In [3]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [4]:
df.dtypes

orderid                  int64
pick                     int64
1st_deliver_attempt    float64
2nd_deliver_attempt    float64
buyeraddress            object
selleraddress           object
dtype: object

In [5]:
def date_converter(column_name):
    converted = []
    for date in df[column_name]:
        if math.isnan(date):
            converted.append(date)
        else:
            converted.append(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(date))))

    df[column_name] = converted
    df[column_name] = pd.to_datetime(df[column_name])


In [6]:
date_converter('pick')
date_converter('1st_deliver_attempt')
date_converter('2nd_deliver_attempt')

In [8]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [11]:
holidays  = pd.Series(['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
holidays = pd.to_datetime(holidays)

In [24]:
SLA  = []
for index, row in df.iterrows():
    if 'metro manila' in (row['selleraddress']).lower():
        if 'metro manila' in (row['buyeraddress']).lower():
            SLA.append(3)
        elif 'luzon' in (row['buyeraddress']).lower():
            SLA.append(5)
        elif 'visayas' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'mindanao' in (row['buyeraddress']).lower():
            SLA.append(7)
    elif 'luzon' in (row['selleraddress']).lower():
        if 'metro manila' in (row['buyeraddress']).lower():
            SLA.append(5)
        elif 'luzon' in (row['buyeraddress']).lower():
            SLA.append(5)
        elif 'visayas' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'mindanao' in (row['buyeraddress']).lower():
            SLA.append(7)
    elif 'visayas' in (row['selleraddress']).lower():
        if 'metro manila' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'luzon' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'visayas' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'mindanao' in (row['buyeraddress']).lower():
            SLA.append(7)
    elif 'mindanao' in (row['selleraddress']).lower():
        if 'metro manila' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'luzon' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'visayas' in (row['buyeraddress']).lower():
            SLA.append(7)
        elif 'mindanao' in (row['buyeraddress']).lower():
            SLA.append(7)
df['SLA'] = SLA

In [28]:
df.to_csv('cleaned_df.csv', index = False)

# Identifying late deliveries

In [34]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,SLA
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",3
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",3
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",3
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,5


In [90]:
is_late = []
for index, row in df.iterrows():
    
    # Setting the Variables ------------------------
    
    SLA = row['SLA']
    
    firstDays =  (row['1st_deliver_attempt'].date() - row['pick'].date()).days
    
    if str(df['2nd_deliver_attempt'][0]) == 'NaT':
        secondDays = False
    else:
        secondDays =  (row['2nd_deliver_attempt'].date() - row['1st_deliver_attempt'].date()).days
      
    # Subtracting the days delivered according to the rules -----------------------
    
    # Subtracting Holidays
    
    for date in holidays:
        if (row['pick'] <= date <= row['1st_deliver_attempt']):
            firstDays -= 1
            
        if secondDays != False:
            if (row['1st_deliver_attempt'] <= date <= row['2nd_deliver_attempt']):
                secondDays -= 1
            
    # Subtracting Sundays
    
    firstRange = firstDays
    for i in range(firstRange + 1):
        day = row['pick'] + timedelta(days=i+1)
        if day.weekday() == 6:
            firstDays -= 1
            
    if secondDays != False:
        secondRange = secondDays
        for i in range(secondRange + 1):
            day = row['1st_deliver_attempt'] + timedelta(days=i+1)
            if day.weekday() == 6:
                secondDays -= 1
            
    # Checking if the delivery is late or not.
    
    if firstDays <= row['SLA']:
        if secondDays == False:
            is_late.append(0)
        elif secondDays <= 3:
            is_late.append(0)
        else:
            is_late.append(1)
    else:
        is_late.append(1)

In [91]:
print(len(df),len(is_late))

3176313 3176313


In [92]:
results = pd.DataFrame(zip(df['orderid'], is_late))
results.columns = (['orderid','is_late'])

In [93]:
results.to_csv('mago-scl-5-output2.csv', index = False)